In [9]:
import os
from tqdm import tqdm
from collections import defaultdict
from pathlib import Path

In [3]:
error_cons = open('../error.list').readlines()
error_names = {os.path.basename(ele.strip())[:-4]: True for ele in error_cons}

In [12]:
file = '../data/raw/text'
cons = open(file).readlines()
exp_dir = Path('../data/raw_clean')
exp_dir.mkdir(exist_ok=True, parents=True)

with open(exp_dir / os.path.basename(file), 'w') as f:
    # clean_cons = []
    for line in tqdm(cons):
        name = line.split(' ', 1)[0]
        if error_names.get(name, False):
            continue
        # clean_cons.append(line)
        f.write(line)

100%|██████████| 3346328/3346328 [00:04<00:00, 726692.87it/s]


# CommonVoice17 dev

In [8]:
import re
from pathlib import Path
from tqdm import tqdm

audio_dir = Path("/share/datasets/CommonVoice/common_voice_17_0/audio/ru/dev/ru_dev_0_wav")
trans_file = "/share/datasets/CommonVoice/common_voice_17_0/transcript/ru/dev.tsv"

audio_files = audio_dir.glob("*.wav")

name2wavfile = {}

for audio_file in audio_files:
    name = audio_file.stem
    name2wavfile[name] = str(audio_file)

kdir = Path('../data/cv17')
kdir.mkdir(exist_ok=True, parents=True)

with open(kdir / 'text', 'w') as ft, \
    open(kdir / 'wav.scp', 'w') as fw:
    cons = open(trans_file).readlines()[1:]
    for line in tqdm(cons):
        _, name, _, text, *_ = line.strip().split('\t')
        text = re.sub('[.?,!—\–\"\'-;]','', text).strip()
        ft.write(f'{name[:-4]} {text}\n')
        fw.write(f'{name[:-4]} {name2wavfile[name[:-4]]}\n')

100%|██████████| 10203/10203 [00:00<00:00, 58046.41it/s]


# GOLOS Test

In [13]:
import re
import json
from pathlib import Path
from tqdm import tqdm
data_dir = Path('/share/rawdata/golos_opus/test_opus')


kdir = Path('../data/golos')
kdir.mkdir(exist_ok=True, parents=True)


with open(kdir / 'text', 'w') as ft, \
    open(kdir / 'wav.scp', 'w') as fw:
    for folder in data_dir.glob("*"):
        json_file = folder / f'{folder.name}.jsonl'
        print(json_file)
        cons = open(json_file).readlines()
        for line in cons:
            line = json.loads(line)
            name = line['id']
            wav_file = folder / line['audio_filepath']
            text = line['text']
            ft.write(f'{name} {text}\n')
            fw.write(f'{name} {wav_file}\n')

/share/rawdata/golos_opus/test_opus/crowd/crowd.jsonl
/share/rawdata/golos_opus/test_opus/farfield/farfield.jsonl


# RULS


In [14]:
json_file = Path('/share/rawdata/ruls/dev/manifest.json')

kdir = Path('../data/ruls')
kdir.mkdir(exist_ok=True, parents=True)

with open(kdir / 'text', 'w') as ft, \
    open(kdir / 'wav.scp', 'w') as fw:
    cons = open(json_file).readlines()
    for line in cons:
        line = json.loads(line)
        wav_file = json_file.parent / line['audio_filepath']
        name = wav_file.stem
        text = line['text']
        ft.write(f'{name} {text}\n')
        fw.write(f'{name} {wav_file}\n')

# Testset

In [15]:
name2file = {
    'cv17': Path('/data/duhu/ASR-Evaluation/exp/ru_ru__cv17-ru__zentao-16601/2024-11-18/raw/data.json'),
    'dataocean': Path('/data/duhu/ASR-Evaluation/exp/ru_ru__DataOcean-ru__zentao-16601/2024-12-26/raw/data.json'),
    'fleurs': Path('/data/duhu/ASR-Evaluation/exp/ru_ru__fleurs-ru__zentao-16601/2024-11-18/raw/data.json')
}

for name, file in name2file.items():
    kdir = Path(f'../data/{name}')
    kdir.mkdir(exist_ok=True, parents=True)

    with open(kdir / 'text', 'w') as ft, \
        open(kdir / 'wav.scp', 'w') as fw:
        cons = json.load(open(file))

        for line in cons['data']:
            wav_file = line['wav_file']
            name = line['name']
            text = line['ref'].strip()
            ft.write(f'{name} {text}\n')
            fw.write(f'{name} {wav_file}\n')
    